<a href="https://colab.research.google.com/github/ugurcun/Depresyon_durumu_tahmini/blob/main/Depresyon_durumu_tahmini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
df = pd.read_csv('scores.csv')
df.head()

,number,days,gender,age,afftype,melanch,inpatient,edu,marriage,work,madrs1,madrs2
0,condition_1,11,2,35-39,2.0,2.0,2.0,6-10,1.0,2.0,19.0,19.0
1,condition_2,18,2,40-44,1.0,2.0,2.0,6-10,2.0,2.0,24.0,11.0
2,condition_3,13,1,45-49,2.0,2.0,2.0,6-10,2.0,2.0,24.0,25.0
3,condition_4,13,2,25-29,2.0,2.0,2.0,11-15,1.0,1.0,20.0,16.0
4,condition_5,13,2,50-54,2.0,2.0,2.0,11-15,2.0,2.0,26.0,26.0


In [44]:
df.isnull().sum()

,0
number,0
days,0
gender,0
age,0
afftype,32
melanch,35
inpatient,32
edu,2
marriage,32
work,32


In [45]:
# Eksik değerleri doldurma
for column in df.columns:
  if df[column].isnull().any():
    if pd.api.types.is_numeric_dtype(df[column]):
      df[column].fillna(df[column].mean(), inplace=True)
    else:
      df[column].fillna(df[column].mode()[0], inplace=True)

# Kategorik değişkenleri dönüştürme (One-Hot Encoding)
categorical_features = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=categorical_features)

In [46]:
# Veri setini eğitim ve test olarak ayırma
from sklearn.model_selection import train_test_split
X = df.drop(['madrs1','madrs2'], axis=1)  # Pass column names as a list
y = df[['madrs1','madrs2']] # Changed to a list of column names
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Artık X_train, X_test, y_train, y_test değişkenlerini kullanarak modelinizi eğitebilirsiniz.

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Veri setini eğitim, doğrulama ve test olarak ayırma
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2


# Lineer regresyon modeli
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)
y_pred_linear = linear_reg.predict(X_val)

# RMSE hesaplama
rmse_linear = np.sqrt(mean_squared_error(y_val, y_pred_linear))
print(f"Lineer Regresyon RMSE: {rmse_linear}")

# MAE hesaplama
mae_linear = mean_absolute_error(y_val, y_pred_linear)
print(f"Lineer Regresyon MAE: {mae_linear}")

# Random Forest modeli
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train, y_train)
y_pred_rf = rf_reg.predict(X_val)

# RMSE hesaplama
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
print(f"Random Forest RMSE: {rmse_rf}")

# MAE hesaplama
mae_rf = mean_absolute_error(y_val, y_pred_rf)
print(f"Random Forest MAE: {mae_rf}")


# En iyi modeli seçme (RMSE'ye göre)
if rmse_linear < rmse_rf:
  best_model = linear_reg
  print("En iyi model: Lineer Regresyon")
else:
  best_model = rf_reg
  print("En iyi model: Random Forest")


# Test seti üzerinde performans değerlendirmesi
y_pred_test = best_model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f"Test Seti RMSE: {rmse_test}")

# MAE hesaplama
mae_test = mean_absolute_error(y_test, y_pred_test)
print(f"Test Seti MAE: {mae_test}")


Lineer Regresyon RMSE: 4.7456194373456
Lineer Regresyon MAE: 3.4070579738980187
Random Forest RMSE: 4.498821098751338
Random Forest MAE: 3.0730237154150224
En iyi model: Random Forest
Test Seti RMSE: 3.462326424601414
Test Seti MAE: 1.9943675889328096
